# SciServer Science Demo Notebook 1

### Query SDSS Data Release 13, save thumbnails to SciDrive and data to CasJobs

This notebook shows you how to use SciServer Compute to communicate with the other components of SciServer. 

You will learn how to:

1. Import Python libraries to work with SciServer
2. Use CasJobs to query databases - specifically, find galaxies in SDSS DR13
3. Write Python code to plot your data.
4. Store the results of your analysis on a local scratch disk (as an HD5 file) for later reuse.
5. Retrieve thumbnail images for your selected objects from the SDSS ImgCutout service.
6. Save the thumbnails to your SciDrive, where you can easily share them with your colleagues.
7. Save data for these galaxies to your CasJobs MyDB
8. Optional: connect this notebook with your SciServer login portal account (using a Keystone token)

# 1. Import SciServer libraries 
The SciServer team has written a library called <tt>SciServer</tt> to allow Jupyter notebooks like this one to communicate with all SciServer components. The <tt>SciServer</tt> library includes a module for working with each SciServer component. As with all Python modules, they must be actively imported before being used.

The Code Cell below imports those, together with some standard Python libraries helpful for scientific analysis. It then sets some options that you might find helpful.

In [ ]:
# Step 1a: Import Python libraries to work with SciServer

import SciServer.CasJobs            # query with CasJobs
import SciServer.SciDrive           # read/write to/from SciDrive
import SciServer.SkyServer          # read from SkyServer
print('Imported SciServer library')

# Step 1b: Import some other libraries you will need
import pandas                       # useful python library for data handling
import matplotlib.pyplot as plt     # standard Python library for graphing
import urllib                       # parse URLs (needed to generate links to thumbnail images)
import skimage.io                   # image processing library, needed to display thumbnail images
#import numpy as np                            # standard Python lib for math ops - we don't use it in this notebook
#import json                                     # Work with Javascript object notation (.json) files
print('Imported other Python libraries')

# Step 1c: Apply some special settings to the imported libraries
# ensure columns get written completely in notebook
pandas.set_option('display.max_colwidth', -1)
# do *not* show python warnings 
import warnings
warnings.filterwarnings('ignore')
print('Options set')

# 2. Query an astronomy database (SDSS DR13)

The next Code Cell searches the SDSS Data Release 13 database via the CasJobs REST API. The query completes quickly, so it uses CasJobs quick mode.

CasJobs also has an asynchronous mode, which will submit job to a queue and will store the results in a table in your MyDB. If your results are very large, it will store the results in MyScratchDB instead.

Run the code block below to query DR13 via CasJobs and print a table of query results. Try changing some of the query parameters to see how the results change.

Documentation on the SciServer Python libraries can be found at our documentation site at:<br />
www.sciserver.org/docs/

The <tt>SciServer.CasJobs</tt> library returns results as a Pandas DataFrame. Pandas documentation is available at:<br />
http://pandas.pydata.org/pandas-docs/stable/

<font color='red'>TODO:</font> make example with batch query mode.

In [ ]:
# Step 2a: Find objects in the Sloan Digital Sky Survey's Data Release 13.
# Queries the Sloan Digital Sky Serveys' Data Release 13.
# For the database schema and documentation see www.sdss.org/dr13 and http://skyserver.sdss.org/dr13
#
# This query finds "a 4x4 grid of nice-looking galaxies": 
#   galaxies in the SDSS database that have a spectrum 
#   and have a size (petror90_r) larger than 10 arcsec.
# 
# First, store the query in an object called "query" (surround with triple-quotes to avoid parsing errors)
query="""
SELECT TOP 16 p.objId,p.ra,p.dec,p.petror90_r
  FROM galaxy AS p
   JOIN SpecObj AS s ON s.bestobjid = p.objid
WHERE p.u BETWEEN 0 AND 19.6
  AND p.g BETWEEN 0 AND 17  AND p.petror90_r > 10
"""
# Step 2b: Send the query to CasJobs.
# This example uses DR13 as context - the code makes a connection to the DR13 database, then runs the query in quick mode.
#   When the query succeeds, an "OK" message prints.
myGalaxies = SciServer.CasJobs.executeQuery(query, "dr13")
print('OK')

# Step 2c: show results (as a Pandas DataFrame)
myGalaxies

# 3. Make simple plots

Now that we have run the query and stored the results, we can start analyzing the results.

Start by making a simple plot of positions, using the results you found from your CasJobs query in section 2.

The Code Cell below creates two plots. The first is a very simple plot just showing the data with no explanations, useful for quick data exploration. The second includes axis labels and a title, for sharing with your teams. More information about setting plot options is available in the PyPlot documentation at https://matplotlib.org/api/pyplot_api.html

In [ ]:
plt.scatter(myGalaxies['ra'], myGalaxies['dec'])
plt.show() 

plt.scatter(myGalaxies['ra'], myGalaxies['dec'])
plt.xlabel('RA',fontsize=14)
plt.ylabel('Dec',fontsize=14)
plt.title('Positions of 16 galaxies in SDSS DR13')
plt.show() 

# 4. Store results in your container for later use

The next Code Cell saves the data from the "myGalaxies" DataFrame as an HD5 file and as a CSV file.

To see these files, go back to your iPython notebook dashboard (the page from which you opened this notebook). Make sure you are in the <b>persistent</b> folder. You should see your files there. Click on the file names to preview.

In [ ]:
# store result as HDF5 file 
h5store = pandas.HDFStore('GalaxyThumbSample.h5')
h5store['galaxies']=myGalaxies
h5store.close()
print('Saved as .h5 file')

# store result as CSV file
myGalaxies.to_csv('GalaxyThumbSample.csv')
print('Saved as .csv file')

# 5. Retrieve thumbnail cutouts of galaxies and show them on screen
SkyServer has a service that will produce a color image cutout of certain dimensions around a specified position, displayed as a JPG thumbnail. (The service creates the thumbnail using a pre-defined image pyramid.)

To print a thumbnail image for a single galaxy, you can construct the URL of the service using your query results (using the <tt>urllib</tt> library), then use the <tt>skimage</tt> package to call the URL. To get all thumbnails for all the galaxies in your query result, you can iterate using a loop.

The Code Cell below gives an example of how to retrieve JPG thumbnails of the 16 galaxies you selected from SDSS DR13.

Note that to display the SDSS objID as a title above each thumbnail, we set the index of the DataFrame to be objID.

The SDSS ImgCutout service is documented at http://skyserver.sdss.org/dr13/en/tools/chart/chartinfo.aspx

In [ ]:
myGalaxies = myGalaxies.set_index('objId')    # set the index of the DataFrame for easy reference later
# set thumbnail parameters
width=200           # image width
height=200          # height
pixelsize=0.396     # image scale
plt.figure(figsize=(15, 15))   # display in a 4x4 grid
subPlotNum = 1

i = 0
nGalaxies = len(myGalaxies)
for index,gal in myGalaxies.iterrows():           # iterate through rows in the DataFrame
    i = i + 1
    print('Getting image '+str(i)+' of '+str(nGalaxies)+'...')
    if (i == nGalaxies):
        print('Plotting images...')
    scale=2*gal['petror90_r']/pixelsize/width
    img= SciServer.SkyServer.getJpegImgCutout(ra=gal['ra'], dec=gal['dec'], width=width, height=height, scale=scale,dataRelease='DR13')
    plt.subplot(4,4,subPlotNum)
    subPlotNum += 1
    plt.imshow(img)                               # show images in grid
    plt.title(index)                              # show the object identifier (objId) above the image.

# 6. Write thumbnails to SciDrive

SciDrive allows you to save query results as flat files in a Dropbox-like interface you can access anywhere. The Code Cell below saves the thumbnail images you generated in step 5 to your SciDrive. The first line allows you to change the SciDrive directory into which you write the files.

In [ ]:
directory = 'thumbnails'    # directory to save thumbnail images into

# check whether directory exists; if so, print warning that files may be overwritten.
ok_to_create = True
dirList = SciServer.SciDrive.directoryList("")
for i in dirList['contents']:
    if(i['path'][1:len(i['path'])] == directory):
        ok_to_create = False
if (ok_to_create):
    response = SciDrive.createContainer(directory)
else:
    print('Warning! Directory already exists. You might overwrite files in your existing directory.')

# Get thumbnails as in step 5
width=200
height=200
pixelsize=0.396

filelocs=[]

i = 0
for index,gal in myGalaxies.iterrows():
    i = i + 1
    scale=2*gal['petror90_r']/pixelsize/width
    url="http://skyserver.sdss.org/DR13/SkyServerWS/ImgCutout/getjpeg.aspx?ra="+str(gal['ra'])
    url+="&dec="+str(gal['dec'])+"&scale="""+str(scale)+"&width="+str(width)
    url+="&height="+str(height)
    req = urllib.urlopen(url)
    data = req.read()
    scidrivename_name = '/' + directory +'/new_'+str(index)+'.jpg'
    filelocs.append(scidrivename_name)                                               # save filename for future reference
    responseUpload = SciServer.SciDrive.upload(path=scidrivename_name,data=data)     # write file to SciDrive
    print('Saving thumbnail '+str(i)+' of '+str(len(myGalaxies))+': objID = '+str(index))
print('Done!')

Now check your SciDrive again (<a href='www.scidrive.org'>www.scidrive.org</a>). You should see a directory called "thumbnails" (or whatever you chose above).

Double-click on the directory name to open. You should see the thumbnails you just saved.

# 7. Store all results in a table in your MyDB

So far, you have found data about some galaxies in SDSS Data Release 13. In addition to the data that DR13 provides, you have generated thumbnail images for each galaxy. You have also used the SciServer.SciDrive module to save the thumbnails to your SciDrive.

Next, you will store all this information in a new table in your CasJobs personal database space, your <tt>MyDB</tt>.

Remember that all the data you queried from DR13 is now stored in a Pandas DataFrame called "myGalaxies". You'll be starting from that data frame to save your results in your MyDB. You will also add a column giving the link to the thumbnail image in your SciDrive.

You can see what is currently in your MyDB from the CasJobs website:<br />http://skyserver.sdss.org/casjobs/

In [ ]:
newtablename = 'GalaxyThumbs3'   # the name of the new table you will create in your MyDB

# Step 7a: First, add another column to the data frame to give a link to the thumbnails you saved into SciDrive.
puburls = []
for i in range(0,len(filelocs)):
    puburls.append(SciServer.SciDrive.publicUrl(filelocs[i]))   # look up SciDrive's public URL for each of your files
myGalaxies['pubURL'] = puburls

# Step 7b: Create the new table (print a warning and drop if it already exists)
ddl = """CREATE TABLE """ + newtablename + """(objId bigint, ra real, dec real, petror90_r real, pubURL varchar(128))"""
for j in SciServer.CasJobs.getTables('MyDB'):
    if(j['Name'] == newtablename):
        print('Warning! Table already exists, will be dropped and re-created.')
        ddl = """DROP TABLE """ + newtablename
        SciServer.CasJobs.executeQuery(ddl,'MyDB')

# Step 7c: Upload your DataFrame to the newly-created table
SciServer.CasJobs.uploadPandasDataFrameToTable(myGalaxies,newtablename,"MyDB")

# Step 7d: Query your MyDB to show the newly-added table
query2 = '''select * from '''+newtablename
print('Your new table: '+newtablename)
SciServer.CasJobs.executeQuery(query2, "MyDB")

# 8. Get your SciServer token (optional)

SciServer libraries automatically log you in through the SciServer Portal, so you no longer need to worry about authentication.

But sometimes it is useful to know your SciServer token. Run the code in the Code Cell below to print your token, along with your SciServer username. You can use this to log in to the SciServer system if you ever need to use it from an outside container.

In [ ]:
# This code block defined your token and makes it available as a 
#   system variable for the length of your current session.
# 
# This will usually be the first code block in any script you write.

import SciServer.Authentication as Auth
token=Auth.getToken()
user = Auth.getKeystoneUserWithToken(token)
print("Your username is: "+user.userName)
print("Your current token is"+token)